In [9]:
%pip install xgboost
%pip install catboost

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

In [11]:
url = "http://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data"
columns = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
df = pd.read_csv(url, names=columns)
df.head()

,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [12]:
df_encoded = pd.get_dummies(df, columns=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety'])
X = df_encoded.drop('class', axis=1)
y = df_encoded['class']

encoder = LabelEncoder()
y_trans = encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_trans, test_size=0.2, random_state=32)

In [13]:
xgb_classifier = XGBClassifier(objective='multi:softmax')
xgb_classifier.fit(X_train, y_train)
y_pred_xgb = xgb_classifier.predict(X_test)

print("XGBoost Classification Report:")
print(classification_report(y_test, y_pred_xgb))

XGBoost Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99        77
           1       1.00      1.00      1.00        12
           2       1.00      1.00      1.00       245
           3       1.00      1.00      1.00        12

    accuracy                           1.00       346
   macro avg       1.00      1.00      1.00       346
weighted avg       1.00      1.00      1.00       346



In [14]:
cat_classifier = CatBoostClassifier(iterations=150, depth=5, learning_rate=0.3, loss_function='MultiClass', verbose=15)
cat_classifier.fit(X_train, y_train)
y_pred_cat = cat_classifier.predict(X_test)

print("CatBoost Classification Report:")
print(classification_report(y_test, y_pred_cat))

0:	learn: 0.9716852	total: 2.21ms	remaining: 329ms
15:	learn: 0.2321337	total: 35.7ms	remaining: 299ms
30:	learn: 0.1205501	total: 66.7ms	remaining: 256ms
45:	learn: 0.0782049	total: 109ms	remaining: 247ms
60:	learn: 0.0604741	total: 134ms	remaining: 196ms
75:	learn: 0.0467108	total: 158ms	remaining: 154ms
90:	learn: 0.0382439	total: 182ms	remaining: 118ms
105:	learn: 0.0322731	total: 207ms	remaining: 86ms
120:	learn: 0.0276432	total: 232ms	remaining: 55.5ms
135:	learn: 0.0243981	total: 256ms	remaining: 26.4ms
149:	learn: 0.0214798	total: 279ms	remaining: 0us
CatBoost Classification Report:
              precision    recall  f1-score   support

           0       0.97      1.00      0.99        77
           1       1.00      1.00      1.00        12
           2       1.00      0.99      1.00       245
           3       1.00      1.00      1.00        12

    accuracy                           0.99       346
   macro avg       0.99      1.00      1.00       346
weighted avg       0.9

In [15]:
accuracy_xgb = (y_test == y_pred_xgb).mean()
accuracy_cat = (y_test == y_pred_cat).mean()

print(f"Overall accuracy")
print(f"XGBoost: {accuracy_xgb}")
print(f"CatBoost: {accuracy_cat}")

Overall accuracy
XGBoost: 0.9971098265895953
CatBoost: 0.5505195629656855
